Import required libraries

In [1]:
import numpy as np
from random import random
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue, SimpleQueue, LifoQueue
from tqdm.auto import tqdm
from math import ceil

C:\Users\lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define useful functions

In [2]:
State = namedtuple('State', ['taken', 'not_taken'])

def goal_check(state):
    return np.all(covered(state))

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)])
    )

def show_frontier(frontier: PriorityQueue):
    elements = list()

    print("\nFrontier: ")
    print('--------------------------------------')
    print('State\t\tPriority')

    while not frontier.empty():
        priority, state = frontier.get()
        elements.append((priority, state))
        print(state.taken, '\t\t', priority,)
        
    print('--------------------------------------')


    for element in elements:
        frontier.put(element)

Define problem variables and check if the problem is feasible

In [97]:
PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

**Remarkable example(s)**\
Using these examples to test A* implementation. Greedy best first does not work with these examples.

In [198]:
PROBLEM_SIZE = 6
NUM_SETS = 4
SETS = tuple(
    [np.array([True, True, True, False, False, False]),
    np.array([True, False, False, False, False, True]),
    np.array([False, True, False, False, True, False]),
    np.array([False, False, True, True, False, False])]
)
State = namedtuple('State', ['taken', 'not_taken'])

In [54]:
PROBLEM_SIZE = 6
NUM_SETS = 3
SETS = tuple(
    [np.array([True, True, True, True, False, False]),
    np.array([True, True, False, False, False, True]),
    np.array([False, False, True, True, True, False])]
)
State = namedtuple('State', ['taken', 'not_taken'])

**Depth First Search**

In [84]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action}
            )
            frontier.append(new_state)
        current_state = frontier.pop()
        pbar.update(1)

print(f'Solved in {counter} steps ({len(current_state.taken)}) tiles')
print(f'Tiles taken:', current_state.taken)

20it [00:00, 27351.18it/s]

Solved in 20 steps (20) tiles
Tiles taken: {20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39}


**Breadth First Search**\
Guarantees to find the best solution.

In [28]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.popleft()

with  tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state.not_taken:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action}
                )
            frontier.append(new_state)
        current_state = frontier.popleft()
        pbar.update(1)
    
print(f'Solved in {counter} steps ({len(current_state.taken)}) tiles')
print(f'Tiles taken:', current_state.taken)

0it [00:00, ?it/s]

7968it [00:00, 14942.43it/s]

Solved in 7968 steps (4) tiles
Tiles taken: {16, 8, 3, 0}


**Greedy Best First**\
Finds a good solution in a limited number of steps. Does not guarantee to find the best solution. Needs a function to define how far the current state is from the solution.

In [19]:
def distance(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [20]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0

_, current_state = frontier.get()

with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1

        for action in current_state.not_taken:
            new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
            frontier.put((distance(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f'Solved in {counter} steps ({len(current_state.taken)}) tiles')
print(f'Tiles taken:', current_state.taken)

4it [00:00, ?it/s]

Solved in 4 steps (4) tiles
Tiles taken: {0, 1, 12, 17}


**A***\
Complete and optimally efficient solution. Needs to the find a cost function and a heuristic. The heuristic must be **optimistic**.

Heuristics proposed by the professor

In [106]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(covered(state))
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)

    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

Heuristics proposed by me

In [91]:
# actually does not work, not optimistic
def h4(state):
    """Considers the tile that covers the maximum number of uncovered elements as the next tile taken
       and estimates the number of remaining tiles to take through h3. Actually not optimistic.
    """
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    best_candidate_set = np.argmax(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    probable_next_state = State(state.taken ^ {best_candidate_set}, state.not_taken ^ {best_candidate_set})

    return h3(probable_next_state) + 1

def h5(state):
    """Applies the same idea of h3 to h: sorts the tiles not taken by decreasing order (without considering uncovered elements)
       to estimate the number of remaining tiles.
       Actually perform worse than h.
    """
    candidates = sorted((sum(SETS[i]) for i in state.not_taken), reverse=True)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h6(state):
    """Slight improvement in terms of computational efficience w.r.t. h2: instead of considering all the sets as candidates,
       just considerates the ones not taken.
    """
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(SETS[i], np.logical_not(already_covered))) for i in state.not_taken)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def h7(state):
    """Slight improvement in terms of computational efficience w.r.t. h3: instead of considering all the sets as candidates,
       just considerates the ones not taken. (Same improvement as h6)
    """
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(covered(state))
    candidates = sorted((sum(np.logical_and(SETS[i], np.logical_not(already_covered))) for i in state.not_taken), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h8(state):
    """Estimates the remaining number of tiles (through h2) after adding a candidate to the solution.
       In other words takes one step in the future. In this way the number of steps required by A* should
       decrease, but the number of inner steps (inside the heuristic) increases.

    """
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    
    candidates = [sum(np.logical_and(SETS[i], np.logical_not(already_covered))) for i in state.not_taken]
    min_taken = PROBLEM_SIZE

    for candidate in candidates:
        probable_next_state = State(state.taken ^ {candidate}, state.not_taken ^ {candidate})
        taken = h2(probable_next_state)
        if taken < min_taken:
            min_taken = taken

    return min_taken + 1

def h9(state):
    """Same as h8 but uses h3 to estimate the future remaining numer of tiles.
       Estimates the remaining number of tiles (through h3) after adding a candidate to the solution.
       In other words takes one step in the future. In this way the number of steps required by A* should
       decrease, but the number of inner steps (inside the heuristic) increases.
       Less number of outer iterations (steps required by A*) but higher number of inner iterations
       (steps required by h9) w.r.t. h8.

    """
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    
    candidates = [sum(np.logical_and(SETS[i], np.logical_not(already_covered))) for i in state.not_taken]
    min_taken = PROBLEM_SIZE

    for candidate in candidates:
        probable_next_state = State(state.taken ^ {candidate}, state.not_taken ^ {candidate})
        taken = h3(probable_next_state)
        if taken < min_taken:
            min_taken = taken

    return min_taken + 1

Cost function proposed by the professor

In [9]:
def c(state):
    return len(state.taken)

In [10]:
def f(state, cost_function, heuristic_function):
    return cost_function(state) + heuristic_function(state)

Set  <mark>verbose=True</mark> to understand how the priority queue behaves. Warning: just do it on small instances of the problem.

In [114]:
cost_function = lambda state: c(state)
heuristic_function = lambda state: h8(state)

In [115]:
verbose = False

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state, cost_function, heuristic_function), state))

counter = 0

_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1

    for action in current_state.not_taken:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put((f(new_state, cost_function, heuristic_function), new_state))
    
    if verbose:
        show_frontier(frontier)
        print('\nThe current state is:', current_state)
        
    _, current_state = frontier.get()

print(f'Solved in {counter} steps ({len(current_state.taken)}) tiles')
print(f'Tiles taken:', current_state.taken)

Solved in 111 steps (4) tiles
Tiles taken: {9, 2, 11, 15}
